In [1]:
!pip install eli5

     |████████████████████████████████| 112kB 5.0MB/s 


In [0]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook

In [3]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix"

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [4]:
ls

data/       HelloGithub.ipynb  matrix_one/  tday5.ipynb
day4.ipynb  LICENSE            README.md


In [5]:
ls data

men_shoes.csv


In [0]:
df=pd.read_csv('data/men_shoes.csv',low_memory=False)

In [53]:
df.shape

(18280, 48)

In [0]:
def run_model(feats,model=DecisionTreeRegressor(max_depth=5)):
  X=df[feats].values
  y=df['prices_amountmin'].values

  scores=cross_val_score(model,X,y,scoring='neg_mean_absolute_error')
  return np.mean(scores),np.std(scores)

In [55]:
df['brand_cat']=df['brand'].map(lambda x:str(x).lower()).factorize()[0]
run_model(['brand_cat'])

(-58.133398968282776, 4.206122611474276)

In [56]:
model=RandomForestRegressor(max_depth=5,n_estimators=100,random_state=0)
run_model(['brand_cat'],model)


(-57.31783843165656, 4.181246596160967)

In [57]:
df.features.head().values

array(['[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SERVUS BY HONEYWELL"]},{"key":"manufacturer_part_number","value":["ZSR101BLMLG"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SER

In [0]:
def parse_features(x):
  output_dict={}
  if str(x)=='nan':return output_dict
  features=literal_eval(x.replace('\\"','"'))
  for item in features:
    key=item['key'].lower().strip()
    value=item['value'][0].lower().strip()

    output_dict[key]=value
  
  return output_dict 

df['features_parsed']=df['features'].map(parse_features)

In [59]:
keys=set()


df.features_parsed.map(lambda x:keys.update(x.keys()))

len(keys)

476

In [60]:
def get_name_feat(key):
  return 'feat_'+key 

for key in tqdm_notebook(keys):
  df[get_name_feat(key)]=df.features_parsed.map(lambda feats: feats[key] if key in feats else np.nan)

In [61]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_designer', 'feat_number of heat settings', 'feat_is waterproof',
       'feat_case type', 'feat_wash', 'feat_manufacturer_part_number',
       'feat_materials', 'feat_lining', 'feat_ul safety listing',
       'feat_features'],
      dtype='object', length=526)

In [0]:
keys_stat={}

for key in keys:
  keys_stat[key]=df[False==df[get_name_feat(key)].isnull()].shape[0]/df.shape[0]*100

In [63]:
keys_stat

{'100% adidas authentic': 0.005470459518599562,
 'accessory type': 0.1422319474835886,
 'adidas': 0.005470459518599562,
 'adjustable': 0.34463894967177244,
 'age': 0.6400437636761488,
 'age end': 0.4431072210065646,
 'age gender group': 0.12582056892778995,
 'age group': 27.64770240700219,
 'age range': 0.5470459518599562,
 'age segment': 0.1422319474835886,
 'age start': 0.4431072210065646,
 'airport friendly': 0.005470459518599562,
 'alarm': 0.23522975929978115,
 'amazonbestsellersrank': 0.06017505470459519,
 'animal type': 0.005470459518599562,
 'antiscratch lens coating': 0.005470459518599562,
 'applicable': 0.02188183807439825,
 'arm': 0.6236323851203501,
 'article': 0.005470459518599562,
 'assembled in country of origin': 2.199124726477024,
 'assembled product dimensions (l x w x h)': 10.300875273522976,
 'assembled product weight': 1.5536105032822756,
 'athlete': 0.0437636761487965,
 'atpv arc rating': 0.005470459518599562,
 'attachment': 0.03282275711159737,
 'audience': 0.2352

In [64]:
{k:v for k,v in keys_stat.items() if v>30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
df['feat_brand_cat']=df['feat_brand'].factorize()[0]
df['feat_color_cat']=df['feat_color'].factorize()[0]
df['feat_gender_cat']=df['feat_gender'].factorize()[0]
df['feat_manufacturer part number_cat']=df['feat_manufacturer part number'].factorize()[0]
df['feat_material_cat']=df['feat_material'].factorize()[0]


df['feat_sport_cat']=df['feat_sport'].factorize()[0]
df['feat_style_cat']=df['feat_style'].factorize()[0]

for key in keys:
  df[get_name_feat(key)+'_cat']=df[get_name_feat(key)].factorize()[0]

In [67]:
df[df.brand!=df.feat_brand].shape

(18228, 531)

In [68]:
df[df.brand!=df.feat_brand][['brand','feat_brand']].head()

,brand,feat_brand
0,Josmo,josmo
1,Josmo,josmo
2,SERVUS BY HONEYWELL,servus by honeywell
3,SERVUS BY HONEYWELL,servus by honeywell
4,SERVUS BY HONEYWELL,servus by honeywell


In [0]:
df.brand=df.brand.map(lambda x:str(x).lower())

In [71]:
df[df.brand!=df.feat_brand][['brand','feat_brand']].head()

,brand,feat_brand
21,rubies,generic
22,rubies,generic
23,rubies,generic
24,unbranded,NaN
31,american fighter,NaN


In [72]:
df[df.brand==df.feat_brand].shape

(8846, 531)

In [73]:
model=RandomForestRegressor(max_depth=5,n_estimators=100)
run_model(['brand_cat'],model)

(-57.38968449784204, 4.231086449146932)

In [94]:
feats_cat=[x for x in df.columns if 'cat' in x]
feats_cat

['categories',
 'brand_cat',
 'feat_clothing category',
 'feat_location - country',
 'feat_recommended location',
 'feat_multi pack indicator',
 'feat_catalog',
 'feat_fabrication',
 'feat_certifications and listings',
 'feat_location - city/state',
 'feat_shoe category',
 'feat_brand_cat',
 'feat_color_cat',
 'feat_gender_cat',
 'feat_manufacturer part number_cat',
 'feat_material_cat',
 'feat_sport_cat',
 'feat_style_cat',
 'feat_movement_cat',
 'feat_flame resistant_cat',
 'feat_bag size range_cat',
 'feat_is dark sky-compliant_cat',
 'feat_labor_cat',
 'feat_insulation_cat',
 'feat_product in inches (l x w x h)_cat',
 'feat_hat style_cat',
 'feat_sku_cat',
 'feat_band color_cat',
 'feat_msrp_cat',
 'feat_stylenumber_cat',
 'feat_dial markers_cat',
 'feat_waterproof_cat',
 'feat_band material_cat',
 'feat_label_cat',
 'feat_features & fastening_cat',
 'feat_fits most screen size_cat',
 'feat_upper material_cat',
 'feat_chest_cat',
 'feat_product #_cat',
 'feat_autographed_cat',
 'fe

In [0]:
feats=['brand_cat','feat_gender_cat','feat_shape_cat','feat_material_cat','feat_brand_cat','feat_style_cat','feat_sport_cat']
#feats+=feats_cat

#feats=list(set(feats))

model=RandomForestRegressor(max_depth=5,n_estimators=100)
result=run_model(feats,model)

In [111]:


X=df[feats].values
y=df['prices_amountmin'].values

m=RandomForestRegressor(max_depth=5,n_estimators=100,random_state=0)
m.fit(X,y)
print(result)
perm=PermutationImportance(m,random_state=1).fit(X,y)
eli5.show_weights(perm,feature_names=feats)

(-57.149385237106436, 4.306310479706156)


Weight,Feature
0.2632 ± 0.0109,brand_cat
0.1061 ± 0.0073,feat_material_cat
0.0239 ± 0.0024,feat_gender_cat
0.0222 ± 0.0016,feat_brand_cat
0.0133 ± 0.0011,feat_shape_cat
0.0040 ± 0.0007,feat_style_cat
0.0003 ± 0.0000,feat_sport_cat


In [83]:
df['brand'].value_counts(normalize=True)

nike             0.097210
puma             0.033315
ralph lauren     0.028775
vans             0.021116
new balance      0.020295
                   ...   
ubisoft          0.000055
wings & horns    0.000055
mens             0.000055
slipperooz       0.000055
sansha           0.000055
Name: brand, Length: 1732, dtype: float64

In [84]:
df[df['brand']=='nike'].features_parsed.head().values

array([{'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'style': 'athletic sneakers', 'condition': 'new with box'}, {}],
      dtype=object)

In [91]:
df[df['brand']=='nike'].features_parsed.sample(5).values

array([{'style': 'windbreaker', 'material': 'polyester', 'country/region of manufacture': 'china', 'condition': 'new with tags'},
       {'sport': 'baseball & softball', 'main color': 'white'},
       {'sport': 'lifestyle', 'occasion': 'lifestyle', 'material': 'suede', 'gender': 'men', 'shoe size': '7.5 us men', 'shoe category': "men's shoes", 'assembled product dimensions (l x w x h)': '12.00 x 8.00 x 6.00 inches', 'fabric content': 'suede', 'color': 'wolf grey/white/mtllc silver', 'model': '819720 010', 'shoe closure': 'lace-up', 'casual & dress shoe style': 'fashion sneakers', 'manufacturer part number': '819720 010', 'brand': 'nike', 'age group': 'men', 'shoe width': 'd(m)', 'variant group id': '224#mp#819720 010'},
       {'style': 'running, cross training', 'material': 'synthetic', 'condition': 'new with defects'},
       {'style': 'athletic sneakers', 'country of manufacture': 'china', 'condition': 'new with box'}],
      dtype=object)

In [92]:
df['feat_age group'].value_counts()

adult               4563
men                  350
child                 77
men's                 33
unisex                 6
toddler                4
infant                 4
mens                   4
boys'                  3
women ,�� unisex       2
youth                  2
men||women             2
women                  2
adult ,�� teen         1
12 up                  1
Name: feat_age group, dtype: int64